In [1]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)

True

In [2]:
!wget -O golden_hymns_of_epictetus.txt https://www.guttenberg.org/cache/epub/871/pg871.txt

--2024-04-29 14:41:23--  https://www.guttenberg.org/cache/epub/871/pg871.txt
Resolving www.guttenberg.org (www.guttenberg.org)... 199.59.243.225
Connecting to www.guttenberg.org (www.guttenberg.org)|199.59.243.225|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1086 (1,1K) [text/html]
Saving to: ‘golden_hymns_of_epictetus.txt’

golden_hymns_of_epi 100%[===================>]   1,06K  --.-KB/s    in 0,003s  

2024-04-29 14:41:24 (407 KB/s) - ‘golden_hymns_of_epictetus.txt’ saved [1086/1086]



In [7]:
filename_in = 'golden_hymns_of_epictetus.txt'
filename_out = 'out_golden_hymns_of_epictetus.txt'
start_saving = False
stop_saving = False

lines_to_save = []

with open(filename_in, 'r') as file:
    for line in file:
        if 'Are these the only works of Providence within us?' in line:
            start_saving = True
        if '*** END OF THE PROJECT GUTTENBERG EBOOK THE GOLDEN' in line:
            stop_saving = True
            break
        if start_saving and not stop_saving:
            lines_to_save.append(line)

# Scrittura delle righe in un nuovo file

with open(filename_out, 'w') as file:
    for line in lines_to_save:
        file.write(line)

In [8]:
word_count = 0
with open(filename_out, 'r') as file:
    for line in file:
        words = line.split()
        word_count += len(words)
        
print(f'Il numero di words nel file è {word_count}')

Il numero di words nel file è 26657


Utilizzo un TextLoader al fine di convertire un file di testo in un formato adatto all'utilizzo da parte di un llm

In [10]:
from langchain.document_loaders import TextLoader

loader = TextLoader(filename_out)
golden_saying = loader.load()

In [11]:
type(golden_saying)

list

In [12]:
type(golden_saying[0])

langchain_core.documents.base.Document

Oltre a Textloader ci sono parecchi altri document loader in langchain.<br>
Gli step per utilizzarli sono sempre:
- scegliere un Document Loader adatto alle esigenza
- creare una istanza 
- utilizzare il metodo load() per convertire i files in langchain documents

Per dividere il testo in parti più ridotte e gestibili si utilizza il TEXT SPLITTER.<br>
Il questo caso utilizziamo il RecursiveCharacterTextSplitter

In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=50,
    length_function=len,
    add_start_index=True
    
)

In [25]:
text = text_splitter.split_documents(golden_saying)

In [26]:
print(text[0])
print(text[1])

page_content='Are these the only works of Providence within us? What words suffice to\npraise or set them forth? Had we but understanding, should we ever cease\nhymning and blessing the Divine Power, both openly and in secret, and\ntelling of His gracious gifts? Whether digging or ploughing or eating,\nshould we not sing the hymn to God:‐‐\nGreat is God, for that He hath given us such instruments to till\nthe ground withal: Great is God, for that He hath given us hands and\nthe power of swallowing and digesting; of unconsciously growing and\nbreathing while we sleep!\nThus should we ever have sung; yea and this, the grandest and divinest\nhymn of all:‐‐\nGreat is God, for that He hath given us a mind to apprehend these\nthings, and duly to use them!\nWhat then! seeing that most of you are blinded, should there not be some\none to fill this place, and sing the hymn to God on behalf of all\nmen? What else can I that am old and lame do but sing to God? Were I' metadata={'source': 'out_gol

Dopo aver suddiviso il testo iniziale in parti ridotte, è possibile procedere con la coversione dei testi in <B>EMBEDDINGS<B>

In [29]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
        api_key='545454545',
        base_url='http://localhost:8000/v1',
        chunk_size = 1)

In [30]:
from langchain.vectorstores import FAISS

vectorsore = FAISS.from_documents(documents=text, embedding=embeddings)

InternalServerError: Error code: 500 - {'error': {'message': "[{'type': 'string_type', 'loc': ('body', 'input', 'str'), 'msg': 'Input should be a valid string', 'input': [[11787, 1521, 279, 1193, 4375, 315, 58941, 2949, 603, 30, 3639, 4339, 77256, 311, 198, 652, 4105, 477, 743, 1124, 13544, 30, 24805, 584, 719, 8830, 11, 1288, 584, 3596, 32616, 198, 71, 1631, 1251, 323, 40149, 279, 43361, 7572, 11, 2225, 30447, 323, 304, 6367, 11, 323, 198, 83, 6427, 315, 5414, 82426, 21258, 30, 13440, 42200, 477, 628, 1409, 287, 477, 12459, 345, 5562, 584, 539, 7936, 279, 67743, 77, 311, 4359, 25, 31021, 31021, 198, 22111, 374, 4359, 11, 369, 430, 1283, 52677, 2728, 603, 1778, 24198, 311, 12222, 198, 1820, 5015, 449, 278, 25, 8681, 374, 4359, 11, 369, 430, 1283, 52677, 2728, 603, 6206, 323, 198, 1820, 2410, 315, 91747, 323, 21552, 287, 26, 315, 26118, 71888, 7982, 323, 198, 21152, 44661, 1418, 584, 6212, 4999, 45600, 1288, 584, 3596, 617, 40439, 26, 379, 12791, 323, 420, 11, 279, 6800, 478, 323, 3512,

In [34]:
!pip install sentence_transformers

  Using cached sentence_transformers-2.7.0-py3-none-any.whl.metadata (11 kB)
  Using cached transformers-4.40.1-py3-none-any.whl.metadata (137 kB)
  Using cached torch-2.3.0-cp312-cp312-manylinux1_x86_64.whl.metadata (26 kB)
  Using cached scikit_learn-1.4.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached scipy-1.13.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached pillow-10.3.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (9.2 kB)
  Using cached sympy-1.12-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
  Using cached Jinja2-3.1.3-py3-none-any.whl.metadata (3.3 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 759.7 kB/s eta 0:00:00m eta 0:00:010:01:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 879.8 kB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 444.7 kB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 767.0 kB/s eta 0:00:00m eta 0:00:010:00:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.2/301.2 kB 486.7 kB/s eta 0:00:001m479.2 kB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 640.8 kB/s eta 0:00:00 kB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 1.0 MB/s eta 0:00:00m eta 0:00:010:00:010m
Using cached Jinja2-3.1.3-py3-none-any.whl (133 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 1.0 MB/s eta 0:00:001.0 MB/s eta 0:00:010m
Using cached sympy-1.12-py3-none-any.whl (5.7 MB)
Using cached MarkupSafe-2.1.5-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (28 kB)
Using 

In [35]:
from langchain_community.embeddings import HuggingFaceEmbeddings

/home/michele/anaconda3/envs/langchain_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [38]:
!pip install faiss-cpu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 987.8 kB/s eta 0:00:00m eta 0:00:010:00:01


In [40]:
from langchain.vectorstores import FAISS

vectorstore = FAISS.from_documents(documents=text, embedding=embeddings_hf)

In [41]:
query = "How can I practice mindfulness if I am always so busy and distracted?"

vectorstore.similarity_search(query)

[Document(page_content='Something of the same sort is true also of diseases of the mind. Behind,\nthere remains a legacy of traces and blisters: and unless these are\neffectually erased, subsequent blows on the same spot will produce\nno longer mere blisters, but sores. If you do not wish to be prone\nto anger, do not feed the habit; give it nothing which may tend its\nincrease. At first, keep quiet and count the days when you were not\nangry: "I used to be angry every day, then every other day: next every\ntwo, next every three days!" and if you succeed in passing thirty days,\nsacrifice to the Gods in thanksgiving.\nLXXVI\nHow then may this be attained?‐‐Resolve, now if never before, to approve\nthyself to thyself; resolve to show thyself fair in God\'s sight; long to\nbe pure with thine own pure self and God!\nLXXVII\nThat is the true athlete, that trains himself to resist such outward\nimpressions as these.\n"Stay, wretched man! suffer not thyself to be carried away!" Great is', me

### Generate

In [43]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

template = """
Use the following pieces of context to answer the question at the end.

If you don't know the answer, just say 'I don't know', don't try to make up an answer.

Use three sentence maximump and keep the answer as concise as possible.

Use the active voice and speak directly to the reader using a concise language.
{context}

Question: {question}

Helpfull Answer:

"""

QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

openai_llm = ChatOpenAI(
        api_key='545454545',
        base_url='http://localhost:8000/v1',
        temperature=0
    )

qa_chain = RetrievalQA.from_chain_type(
    openai_llm,
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt" : QA_CHAIN_PROMPT}
)

query = "What do grief, fear, anvy and desire stem from?"

result = qa_chain.invoke({"query" : query})

result['result']

' Grief, fear, anvy, and desire stem from a lack of understanding or knowledge about the world around us. They are caused by our inability to accept or understand certain situations or events that we encounter in life. These emotions can be overcome by learning more about ourselves and the world around us, and by developing a greater sense of understanding and acceptance. By doing so, we can learn to deal with these emotions in a more constructive way and find peace and happiness in our lives.'

### Utilizzo di LCEL

In [46]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel({"context" : vectorstore.as_retriever(), "question" : RunnablePassthrough()})

chain = setup_and_retrieval | QA_CHAIN_PROMPT | openai_llm | output_parser

chain.invoke(query)

' Grief, fear, anvy, and desire stem from a lack of understanding or acceptance of reality. They are caused by a disconnect between what we want and what we have, or between what we believe is right and what we experience in life. They are also caused by a lack of control over our circumstances or a lack of understanding of how our actions affect our lives. To overcome these negative emotions, we need to develop a deeper understanding of ourselves and the world around us, and learn to accept what we cannot change. We also need to develop a sense of purpose and meaning in life, and learn to focus on what we can control rather than what we cannot. By doing so, we can find peace and happiness in life.'